# Test EV control

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from ev_ssm import ev_ssm
from tqdm import tqdm
import scipy.stats as stats

In [2]:
# manually search perceptron hyperparameters for binary classification
import cProfile as profile
import pstats

In [3]:
# --- Response ---
from ev_ssm import ev_ssm
import numpy as np

sse = ev_ssm(ts=18, N=10000, step=1, tp=40,
             lr=0.1, lp=40, seed=2022, name="EVA",
             n_pref=1, is_report=True,
             tt_mean=0.5, tt_var=0.2, tt_lb=0, tt_ub=1,
             ict_off=False, ecc_off=False)
sse.load_A("Aest.csv")

evd0 = sse.ev.copy()

# quiet some EVs by force
# np.random.seed(sse.config["seed"])
# ict_rid = np.random.randint(low=0,
#                             high=sse.ev.shape[0],
#                             size=3000)

# sse.ev.na.iloc[ict_rid] = sse.ev.nam.iloc[ict_rid]
# sse.ev.lc.iloc[ict_rid] = 1
# sse.g_u()

# initial EV data
evd0 = sse.ev.copy()

sse.g_frc()

EVA: ts=18[H], 10000 EVs, Total Q=87.7 MWh
Online 2925, Q=73.03 MWh, SoC=0.7717
Power(MW): Pt=-7.9463, Pc=-7.9463, Pd=-0.0
Ctrl: 0=1584; 1=1341; 
EVA: Load A from Aest.csv.


[23.84411276428648, 1.8277132850261788]

In [4]:
prof = profile.Profile()
prof.enable()
for i in range(100):
        sse.run(tf=18 + i/3600, Pi=100,
                is_updateA=False, is_rstate=True,
                is_test=False, disable=True)
prof.disable()

EVA: end time 18.0[H] is too close to start time 18[H],simulation will not start.


In [5]:
stats = pstats.Stats(prof).strip_dirs().sort_stats("tottime")
stats.print_stats(20)

         220743471 function calls (220672696 primitive calls) in 73.524 seconds

   Ordered by: internal time
   List reduced from 1149 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
 10998003   15.606    0.000   36.087    0.000 series.py:943(__getitem__)
59898807/59897559    6.173    0.000    7.631    0.000 {built-in method builtins.isinstance}
 11011181    5.196    0.000    8.682    0.000 indexing.py:2486(check_deprecated_indexers)
7792786/7792766    4.470    0.000   13.453    0.000 generic.py:5577(__setattr__)
  3883066    3.982    0.000   20.917    0.000 apply.py:982(series_generator)
 11008449    3.262    0.000    5.210    0.000 series.py:687(_values)
     1111    2.903    0.003   66.495    0.060 apply.py:862(apply_series_generator)
  3897635    2.552    0.000    8.363    0.000 series.py:640(name)
  3881955    2.497    0.000    2.926    0.000 managers.py:1898(set_values)
 11012337    2.108    0.000    3.144    0.000 common.py

In [ ]:
sse.run(tf=18 + 10/3600, Pi=0,
        is_updateA=False, is_rstate=True,
        is_test=False, disable=False)

# sse.run(tf=18 + 100/3600, Pi=0,
#         is_updateA=False, is_rstate=True,
#         is_test=False, disable=False)

In [ ]:
sse.plot_agc()

In [ ]:
sse.plot()

In [ ]:
sse.data

In [ ]:
sse.tsd.plot(y=['Pr', 'Prc', 'Per'])

In [ ]:
sse.tsd

In [ ]:
sse.plot_agc(tu='s')

In [ ]:
sse.run(tf=18 + 10/3600, Pi=0,
        is_updateA=False, is_rstate=True,
        is_test=False, disable=False)

sse.run(tf=18 + 60/3600, Pi=10,
        is_updateA=False, is_rstate=True,
        is_test=False, disable=False)
evd = sse.ev.copy()

In [ ]:
evd0[evd0['lc']==1][['u', 'c', 'c0', 'lc', 'na', 'nam']].head()
evd[evd['lc']==1][['u', 'c', 'c0', 'lc', 'na', 'nam']].head()

# row index of evd0 where lc == 1
lcrid = evd0[evd0['lc'] == 1].index

# compared dataframe
evdc = pd.DataFrame()
evdc['lc0'] = evd0['lc']
evdc['lc1'] = evd['lc']
evdc['c0'] = evd0['c']
evdc['c1'] = evd['c']
evdc['soc0'] = evd0['soc']
evdc['soc1'] = evd['soc']
evdc[evdc['lc0']==1][evdc['c0'] != evdc['c1']]

In [ ]:
# plt.plot(sse.Perl)

In [ ]:
sse.xtab.sum().sum()

In [ ]:
sse.nec

In [ ]:
sse.rtab.sum().sum()

In [ ]:
sse.x0.sum()

In [ ]:
sse.plot(tu='s')

In [ ]:
sse.ev['agc'].sum()

In [ ]:
sse.Prc

In [ ]:
sse.plot_agc(tu='s')

In [ ]:
# --- Response ---

# sse = ev_ssm(ts=10.4, N=10000, step=1, tp=40,
#              lr=0.1, lp=40, seed=2022, name="EVA")
# sse.load_A("Aest.csv")

t_list = [60 + 60 * i for i in list(range(7))]
p_list = [2, 0.05, 0.0, 1.0, -0.05, 1, 0]

x0 = sse.x0.copy()
print(sse.Prc)
for t, p in zip(t_list, p_list):
        sse.run(tf=18 + t/3600, Pi=p,
                is_updateA=False, is_rstate=True,
                is_test=False, disable=True)
print(sse.Prc)
x1 = sse.x0.copy()

In [ ]:
plt.plot(sse.tss, sse.Perl)

In [ ]:
sse.plot_agc()

In [ ]:
sse = ev_ssm(ts=10.4, N=10000, step=1, tp=40,
             lr=0.1, lp=40, seed=2022, name="EVA",
             is_report=False)

# historical data
ev_num = pd.read_csv("ev_num.csv")

# calculate the sf
k1 = ev_num['ne'][(ev_num['time'] >= sse.ts) & (ev_num['time'] <= sse.ts+1/12)].mean()
k0 = ev_num['ne'][ev_num['time'] >= sse.ts].iloc[0]
k = k1 / k0

# estiamte FRC
sse.g_frc(nea=sse.ne*k)

In [ ]:
## Run to get the EV numbers within a day

# import joblib
# from joblib import Parallel, delayed

# Nt = 10000
# def rne(time):
#     sse1 = ev_ssm(ts=time, N=Nt, step=1, tp=40,
#                   lr=0.1, lp=60, seed=2022, name="EVA",
#                   is_report=False)
#     return sse1.ne

# # interval by minutes, 24H
# rnel = Parallel(n_jobs=joblib.cpu_count())(delayed(rne)(i) for i in np.arange(1/60, 24, 1/60))

# ev_num = pd.DataFrame()
# ev_num['time'] = np.arange(1/60, 24, 1/60)
# ev_num['ne'] = rnel
# ev_num.to_csv("ev_num.csv", index=False)